<a href="https://colab.research.google.com/github/oliverZZ/193B_Glucose_Monitor/blob/master/Infection_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
import pandas as pd
pd.options.display.max_rows = 20
import matplotlib.pyplot as plt
import numpy as np

bg = pd.read_excel('/content/drive/My Drive/EEC 193/EEC 193B/Burn_Glucose.xlsx', skiprows = 0)

In [0]:
bg

,PER_CODE,Collection Date,Time Vitals,Systolic,Diastolic,MAP,HR,RR,Temp,CVP,GCS,Vent,Time CBC,WBC,Hgb,Hct,Platelet,Time Labs,Na,K+,BUN,Creatnine,Glucose,Tbili,Chloride,V_CO2,PaO2,FIO2,PaCO2,HCO3,PH,Sepsis
0,21,2011-01-02,614.0,113.0,67.0,78.0,102.0,15.0,37.0,7.0,15.0,No,5.0,6.9,8.1,24.5,578.0,5.0,136.0,3.6,7.0,0.53,110.0,0.3,101.0,29.0,NaN,NaN,NaN,NaN,NaN,0.0
1,21,2011-01-03,600.0,137.0,67.0,85.0,105.0,16.0,37.3,7.0,15.0,No,59.0,7.8,7.9,25.1,615.0,59.0,137.0,3.7,8.0,0.55,130.0,0.3,104.0,27.0,NaN,NaN,NaN,NaN,NaN,0.0
2,21,2011-01-04,1000.0,124.0,56.0,73.0,104.0,18.0,37.5,7.0,15.0,No,208.0,8.9,8.7,26.6,607.0,208.0,135.0,3.4,8.0,0.53,172.0,0.3,103.0,24.0,NaN,NaN,NaN,NaN,NaN,0.0
3,21,2011-01-05,930.0,143.0,69.0,85.0,104.0,18.0,37.5,7.0,15.0,No,103.0,9.3,8.4,25.7,414.0,103.0,134.0,4.8,10.0,0.49,150.0,0.3,103.0,25.0,NaN,0.21,NaN,NaN,NaN,0.0
4,21,2011-01-06,1130.0,130.0,59.0,83.0,104.0,23.0,38.0,7.0,15.0,No,130.0,8.4,8.9,27.7,701.0,130.0,136.0,3.5,11.0,0.51,109.0,0.3,103.0,27.0,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,PCR-004-00017,2013-06-14,600.0,90.0,42.0,59.0,74.0,27.0,36.4,NaN,NaN,Yes,16.0,13.1,9.1,28.0,139.0,16.0,137.0,4.5,17.0,0.90,138.0,NaN,108.0,25.0,NaN,NaN,NaN,NaN,NaN,1.0
747,PCR-004-00017,2013-07-18,600.0,117.0,43.0,67.0,76.0,22.0,36.3,NaN,NaN,Yes,245.0,15.0,7.1,22.8,304.0,245.0,138.0,3.3,50.0,2.09,165.0,NaN,108.0,18.0,173,0.4,43.0,19.0,7.27,1.0
748,PCR-004-00017,2013-07-20,600.0,120.0,47.0,64.0,70.0,18.0,36.6,NaN,NaN,Yes,15.0,6.2,8.6,26.1,154.0,15.0,139.0,3.0,48.0,1.79,135.0,NaN,106.0,23.0,200,0.4,44.0,25.0,7.36,1.0
749,PCR-004-00017,2013-07-23,600.0,121.0,45.0,66.0,80.0,26.0,37.2,NaN,NaN,Yes,103.0,7.6,7.6,22.7,250.0,103.0,142.0,2.3,64.0,1.78,119.0,NaN,102.0,28.0,184,0.4,47.0,28.0,7.4,1.0


## Data Preprocessing ##

In [0]:
def data_preprocess(df):
  # count = 0
  df = df[(df.Sepsis == 0.0) | (df.Sepsis == 1.0)]
  # for i in range(0, len(df)):
  #   if ((df.Sepsis[i] != 0) & (df.Sepsis[i] != 1)):
  df = df.dropna()
  df = df.reset_index(drop = True)
        
  df = df.reset_index(drop = True)
  return df

In [0]:
bg = data_preprocess(bg)

In [0]:
pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.6/dist-packages (0.6.2)


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import imblearn

x = bg[['RR', 'WBC', 'Glucose', 'Platelet']].values
y = bg['Sepsis'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

scaler = RobustScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

balance = imblearn.over_sampling.SMOTE()
train_x, train_y = balance.fit_resample(x_train, y_train)
train_y = pd.Series(train_y)

model = RandomForestClassifier()
model.fit(x_train, y_train)
predictions = model.predict(x_test)

y_pred = model.predict(x_test)

In [0]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.46      0.43      0.44        14
         1.0       0.58      0.61      0.59        18

    accuracy                           0.53        32
   macro avg       0.52      0.52      0.52        32
weighted avg       0.53      0.53      0.53        32

